In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.constants import k as k_B
from scipy.constants import eV as eV
from scipy.optimize import toms748, brentq
from ipywidgets import interactive, FloatSlider, FloatLogSlider, fixed

# Theoretical model

Intrinsic contribution : 

- electrons in conduction band : $n = N_C \exp(-\beta(E_C - \mu))$
- holes in valence band : $p = N_V \exp(-\beta(\mu - E_V))$

Impurities contribution :

- holes in donor level : $h_D = N_D - n_D = N_D - \frac{N_D}{1 + 1/2 \exp(\beta(E_D - \mu))}$
- electrons in acceptor level : $e_A = N_A - n_A = N_A - \frac{N_A}{1 + 1/2 \exp(\mu - E_A))}$

Electrical neutrality equation fixes the chemical potential $\mu$ :

$$n + e_A = p + h_D$$

In [2]:
def get_model(which='lecture'):    
    if which == 'lecture':
        # Approach from the lecture notes
        # Supposing non-degenerate semiconductor,
        # compute intrinsic contributions of holes in valence band $p$,
        # and intrinsic contribution of electrons in conduction band $n$
        n = lambda opts: opts['N_C']*np.exp(-(opts['E_C'] - opts['mu']) / (k_B*opts['T']))
        p = lambda opts: opts['N_V']*np.exp(-(opts['mu'] - opts['E_V']) / (k_B*opts['T']))
        # Compute ionized impurity contributions
        h_D = lambda opts: opts['N_D'] - opts['N_D'] / (1 + 0.5*np.exp((opts['E_D'] - opts['mu']) / (k_B*opts['T'])))
        e_A = lambda opts: opts['N_A'] - opts['N_A'] / (1 + 0.5*np.exp((opts['mu'] - opts['E_A']) / (k_B*opts['T'])))
    
    elif which == 'online':
        # http://ecee.colorado.edu/~bart/book/extrinsi.htm
        # http://ecee.colorado.edu/~bart/book/intrinsi.htm
        n_i = lambda opts: np.sqrt(opts['N_C']*opts['N_V']*np.exp(-(opts['E_C'] - opts['E_V']) / (k_B*opts['T'])))
        E_i = lambda opts: (opts['E_C'] + opts['E_V'])*0.5 + 0.5*k_B*opts['T']*np.log(opts['N_V']/opts['N_C'])
        p = lambda opts: n_i(opts)*np.exp((E_i(opts) - opts['mu']) / (k_B*opts['T']))
        n = lambda opts: n_i(opts)*np.exp((opts['mu'] - E_i(opts)) / (k_B*opts['T']))
        # Compute ionized impurity contributions
        h_D = lambda opts: opts['N_D'] / (1 + 2*np.exp((opts['mu'] - opts['E_D']) / (k_B*opts['T'])))
        e_A = lambda opts: opts['N_A'] / (1 + 4*np.exp((opts['E_A'] - opts['mu']) / (k_B*opts['T'])))
        
    else:
        raise RuntimeError(f'Unknown model {which}')
    
    def eq_neutrality(mu, opts):
        _opts = {**opts, 'mu': mu}
        return h_D(_opts) + p(_opts) - e_A(_opts) - n(_opts)
    
    return n, p, h_D, e_A, eq_neutrality

# Coding...

## Preparing for plotting

In [3]:
def plot_charges_mu(**opts):
    """Demonstrates how the chemical potential solves the neutrality equation"""
    
    opts.update({'mu': np.linspace(-0.5, 1.5, 100)*eV})
    opts['E_D'] *= eV  # convert to Joules
    opts['E_A'] *= eV
    opts['E_C'] *= eV
    n, p, h_D, e_A, eq_neutrality = get_model()
    np.seterr(over='ignore')  # Ignore floating-point overflows
    
    # Compute the chemical potential which ensures charge neutrality
    mu_root = brentq(eq_neutrality, a=min(opts['mu']), b=max(opts['mu']), xtol=eV*1e-3, args=opts)
    
    # Charge neutrality : $N_D - h_D + p = N_A - e_A + n$
    fig, ax = plt.subplots(facecolor='white')
    ax.plot(opts['mu']/eV, h_D(opts) + p(opts))
    ax.plot(opts['mu']/eV, e_A(opts) + n(opts))
    ax.axvline(mu_root/eV, linestyle='--', color='tab:gray')
    ax.set_yscale('log')
    ax.set_ylim((1e0, 1e30))
    ax.set_xlabel('$\\mu$ [eV]')
    ax.set_ylabel('charge carrier density [m$^{{-3}}$]')
    plt.show(fig)
    plt.close(fig)

def plot_mu_T(**opts):
    """Plots the chemical potential as a function of temperature"""
    
    opts.update({'mu': np.linspace(-0.5, 1.5, 100)*eV})
    opts['E_D'] *= eV  # convert to Joules
    opts['E_A'] *= eV
    opts['E_C'] *= eV
    n, p, h_D, e_A, eq_neutrality = get_model()
    np.seterr(over='ignore')  # Ignore floating-point overflows
    
    fig, ax = plt.subplots(facecolor='white')
    Ts = np.linspace(10, 10000, 400)
    # Use a root solver to find the Fermi energy (intersection between the curves)
    mus = np.array([
        brentq(
            eq_neutrality,
            a=min(opts['mu']), b=max(opts['mu']), xtol=eV*1e-3,
            args={**opts, 'T': T}
        )
        for T in Ts
    ])
    ax.plot(Ts*k_B/eV, mus/eV)
    ax.axhline(opts['E_C']/eV, color='tab:gray')
    ax.axhline(opts['E_V']/eV, color='tab:gray')
    ax.axhline(opts['E_D']/eV, linestyle='--', color='tab:gray')
    ax.axhline(opts['E_A']/eV, linestyle='--', color='tab:gray')
    ax.set_xscale('log')
    ax.set_xlabel('$k_B T$ [eV]')
    ax.set_ylabel('$\\mu$ [eV]')
    plt.show(fig)
    plt.close(fig)

def plot_n_T(**opts):
    """Plots conduction electrons as a function of 1/(k_B*T)"""
    
    opts['E_D'] *= eV  # convert to Joules
    opts['E_A'] *= eV
    opts['E_C'] *= eV
    n, p, h_D, e_A, eq_neutrality = get_model()
    np.seterr(over='ignore')  # Ignore floating-point overflows
    
    fig, ax = plt.subplots(facecolor='white')
    Ts = np.linspace(120, 1000, 400)
    # Use a root solver to find the Fermi energy (intersection between the curves)
    mus = np.array([
        brentq(
            eq_neutrality,
            a=-0.5*eV, b=1.5*eV, xtol=eV*1e-3,
            args={**opts, 'T': T}
        )
        for T in Ts
    ])
    _opts = {**opts, 'T': Ts, 'mu': mus}
    ax.plot(1/(Ts*k_B/eV), n(_opts) + e_A(_opts))
    ax.set_yscale('log')
    ax.set_xlabel('$\\frac{1}{k_B T}$ [1/eV]')
    ax.set_ylabel('$n_{tot}$ [eV]')
    plt.show(fig)
    plt.close(fig)

## Preparing interactivity

In [4]:
def get_sliders():
    """Constructs new sliders to vary parameters"""
    
    style = {'description_width': '600px'}
    layout = {'width': '600px'}

    slider_N_D = FloatLogSlider(
        value=1e16, base=10, min=13, max=17, step=0.2,
        description='Number of donors [1/cm³]',
        style=style, continuous_update=False, layout=layout
    )
    slider_E_D = FloatSlider(
        value=0.95, min=0, max=2, step=0.04,
        description='Donor energy [eV]',
        style=style, continuous_update=False, layout=layout
    )
    slider_N_A = FloatLogSlider(
        value=1e14, base=10, min=13, max=17, step=0.2,
        description='Number of acceptors [1/cm³]',
        style=style, continuous_update=False, layout=layout
    )
    slider_E_A = FloatSlider(
        value=0.05, min=0, max=2, step=0.04,
        description='Acceptor energy [eV]',
        style=style, continuous_update=False, layout=layout
    )
    slider_E_V = fixed(
        value=0, min=0, max=2, step=0.04,
        description='Valence band energy [eV]',
        style=style, continuous_update=False, layout=layout
    )
    slider_E_C = FloatSlider(
        value=1.12, min=0, max=2, step=0.04,
        description='Conduction band energy [eV]',
        style=style, continuous_update=False, layout=layout
    )
    slider_T=FloatSlider(
        value=300, min=1, max=500, step=20,
        description='Temperature [K]',
        style=style, continuous_update=False, layout=layout
    )
    slider_N_V = FloatLogSlider(
        value=2.3e19, base=10, min=18, max=20, step=0.2,
        description='Int. VB charge carriers [1/cm³]',
        style=style, continuous_update=False, layout=layout
    )
    slider_N_C = FloatLogSlider(
        value=1e19, base=10, min=18, max=20, step=0.2,
        description='Int. CB charge carriers [1/cm³]',
        style=style, continuous_update=False, layout=layout
    )
    
    return slider_N_D, slider_E_D, slider_N_A, slider_E_A, slider_E_V, slider_E_C, slider_T, slider_N_V, slider_N_C

# Roll the demo !

## Fixing $\mu$

- Change number of donors and number of acceptors to change n-type and p-type
- Effect of temperature (widens the curve)
- Effect of bringing donor band much lower (difficult D.B. to C.B. ionization)
- Effect of bringing acceptor band much higher (difficult V.B to A.B ionization)

In [5]:
slider_N_D, slider_E_D, slider_N_A, slider_E_A, slider_E_V, slider_E_C, slider_T, slider_N_V, slider_N_C = get_sliders()

out = interactive(
    plot_charges_mu,
    N_D=slider_N_D, N_A=slider_N_A,
    N_V=slider_N_V, N_C=slider_N_C,
    E_C=slider_E_C, E_D=slider_E_D, E_A=slider_E_A, E_V=slider_E_V,
    T=slider_T,
)
out.children[-1].layout.height = '300px'
out

interactive(children=(FloatLogSlider(value=1e+16, continuous_update=False, description='Number of donors [1/cm…

## $\mu$ as a function of temperature, and 3 regimes

- Change number of donors and number of acceptors to change n-type and p-type

In [6]:
slider_N_D, slider_E_D, slider_N_A, slider_E_A, slider_E_V, slider_E_C, slider_T, slider_N_V, slider_N_C = get_sliders()

out = interactive(
    plot_mu_T,
    N_D=slider_N_D, N_A=slider_N_A,
    N_V=slider_N_V, N_C=slider_N_C,
    E_C=slider_E_C, E_D=slider_E_D, E_A=slider_E_A, E_V=slider_E_V,
)
out.children[-1].layout.height = '300px'
out

interactive(children=(FloatLogSlider(value=1e+16, continuous_update=False, description='Number of donors [1/cm…

## Electron density in conduction band as a function of temperature

I'm still sceptical for this one, for lower temperatures the graph doesn't look quite right... maybe I did something wrong in the implementation

In [7]:
slider_N_D, slider_E_D, slider_N_A, slider_E_A, slider_E_V, slider_E_C, slider_T, slider_N_V, slider_N_C = get_sliders()

out = interactive(
    plot_n_T,
    N_D=slider_N_D, N_A=slider_N_A,
    N_V=slider_N_V, N_C=slider_N_C,
    E_C=slider_E_C, E_D=slider_E_D, E_A=slider_E_A, E_V=slider_E_V,
)
out.children[-1].layout.height = '300px'
out

interactive(children=(FloatLogSlider(value=1e+16, continuous_update=False, description='Number of donors [1/cm…